<a href="https://colab.research.google.com/github/zjubirdfly/traffic/blob/master/GeneratePicture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Connect to google cloud and import data
import uuid
import pandas as pd
import numpy as np
import os
import glob
import shutil
from google.colab import auth
from google.colab import drive
project_id = 'directed-symbol-252319'
bucket_name = "chongqingtaxidata"
raw_data_bucket_name = "raw_taxi_data"
destination_path = "/content/Taxidata/"
auth.authenticate_user()
drive.mount('/content/drive')
!gcloud config set project {project_id}

In [0]:
#Copy files into system
!mkdir {destination_path}
!gsutil -m cp -r gs://{raw_data_bucket_name}/201703/25 {destination_path}


In [0]:
data_cols = ['date','time', 'label', 'tag','lat','lon','speed?','direction?','status', '?']
files = glob.glob("/content/Taxidata/25/20170325_041.TXT")

all_date_data = []
sample_data = []
for file in files:
    sample_data.append(pd.read_csv(file, sep=',',encoding='GB2312', names= data_cols, header=None,  dtype={'time': object}))

all_date_data = pd.concat(sample_data, axis=0, ignore_index=True)

In [0]:
# Generate N*N graph
N = 25

max_lat = 106.753129
min_lat = 106.273290
lat_range = (max_lat - min_lat) / N

max_lon = 29.878034
min_lon = 29.360739
lon_range = (max_lon - min_lon) / N

graph = np.zeros((N,N))

count = 0
previous_row = None
for index, row in all_date_data.iterrows(): 
  lat = row['lat']
  lon = row['lon']
  if lat > max_lat and lat < min_lat and lon > max_lon and lon < min_lon:
    continue

  #lat index
  lat_index = 0;
  if lat < min_lat:
    lat_index = 0
  elif lat > max_lat:
    lat_index = N - 1
  else:
    lat_index = int((lat - min_lat) / lat_range)
  lat_index = N - 1 - lat_index

  #lon index
  lon_index = 0;
  if lon < min_lon:
    lon_index = 0
  elif lon > max_lon:
    lon_index = N - 1
  else:
    lon_index = int((lon - min_lon) / lon_range)

  if row['status'] == 1:
    count = count + 1
    graph[lon_index, lat_index] = graph[lon_index, lat_index] + 1
  previous_row = row
print(graph)
print(count)

[[3.700e+01 0.000e+00 0.000e+00 0.000e+00 2.000e+00 7.000e+01 1.300e+01
  1.400e+01 0.000e+00 1.000e+01 0.000e+00 1.670e+02 8.600e+01 5.700e+01
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 8.000e+01 0.000e+00 9.900e+01]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  2.900e+01 1.600e+01 1.500e+01 1.200e+01 1.250e+02 2.750e+02 1.460e+02
  1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00
  2.700e+01 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 1.200e+01 2.140e+02 1.000e+01 2.200e+01
  2.700e+01 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 1.300e+01 0.000e+00 1.300e+01]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 9.700e+01 9.200e+01 5.000e+00 1.000e+00
  8.500e+01 3.400e+01 4.000e+01 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000

In [0]:
all_date_data


,date,time,label,tag,lat,lon,speed?,direction?,status,?
0,20170325,032010,H,渝0600314343,106.499977,29.385365,0.0,0,0,1
1,20170325,032000,H,渝0600307706,106.493887,29.620387,0.0,0,0,1
2,20170325,032004,H,渝0600315881,0.000000,0.000000,0.0,0,0,1
3,20170325,032008,H,渝0600314468,106.551173,29.648047,0.0,0,0,1
4,20170325,032041,H,渝0100501505,85.830965,11.924277,35.5,0,1,1
...,...,...,...,...,...,...,...,...,...,...
239864,20170325,032105,H,渝0600307198,106.511753,29.476708,0.0,0,0,1
239865,20170325,032305,H,渝0600307198,106.511758,29.476718,0.0,0,0,1
239866,20170325,032305,H,渝0600307198,106.511758,29.476718,0.0,0,0,1
239867,20170325,032305,H,渝0600307198,106.511758,29.476718,0.0,0,0,1
